# Golf Model

In [9]:
import requests
import csv
from datetime import datetime
import pandas as pd

In [10]:
base_url = "https://feeds.datagolf.com/historical-raw-data/rounds"

api_key = "8742e1e287f26df6a7ecedb8087c"

In [11]:
def get_golf_stats(tour, event_id, year, api_key):
    params = {
        'tour': tour,
        'event_id': event_id,
        'year': year,
        'file_format': 'json',
        'key': api_key
    }
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Define years of interest
years_of_interest = range(2019, datetime.now().year + 1)

# Specify the event_id for the U.S. Open or other major tournaments
event_ids = {'U.S. Open': '535'}  # Add other event IDs as needed

In [14]:
!mkdir -p /mnt/data

mkdir: /mnt: Read-only file system


In [15]:
!ls /mnt/data

ls: /mnt/data: No such file or directory


In [12]:
# Create a file to store the CSV data
with open('/mnt/data/golf_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Year', 'Event ID', 'Event Name', 'Player Name', 'Round', 'Score', 'SG Total', 'Driving Distance', 'GIR', 'Scrambling'])

    # Iterate over each year and event
    for year in years_of_interest:
        for event_name, event_id in event_ids.items():
            # Get the stats for the event
            stats = get_golf_stats('pga', event_id, year, api_key)
            
            # Check if stats is not None
            if stats:
                # Process and write data to CSV
                for player in stats['scores']:
                    for round_number, round_data in player.items():
                        if round_number.startswith('round_'):
                            writer.writerow([
                                year,
                                event_id,
                                event_name,
                                player['player_name'],
                                round_number[-1],  # This gets the round number (e.g., '1' from 'round_1')
                                round_data['score'],
                                round_data['sg_total'],
                                round_data['driving_dist'],
                                round_data['gir'],
                                round_data['scrambling']
                            ])

print("Data collection complete. CSV file created.")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/golf_data.csv'

In [ ]:


# Specify the path to the CSV file
csv_file_path = '/mnt/data/golf_data.csv'

# Now, read the CSV file into a DataFrame
golf_data_df = pd.read_csv(csv_file_path)

# Show the first few rows of the DataFrame
print(golf_data_df.head())